In [99]:
# necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [100]:
import torchvision.transforms as transforms

In [101]:
pip install ucimlrepo

In [102]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
dry_bean_dataset = fetch_ucirepo(id=602)

# data (as pandas dataframes)
X = dry_bean_dataset.data.features
Y = dry_bean_dataset.data.targets


In [103]:
X

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097,759.696,288.721612,185.944705,1.552728,0.765002,42508,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385
13607,42101,757.499,281.576392,190.713136,1.476439,0.735702,42494,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219
13608,42139,759.321,281.539928,191.187979,1.472582,0.734065,42569,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,42147,763.779,283.382636,190.275731,1.489326,0.741055,42667,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222


In [104]:
Y

,Class
0,SEKER
1,SEKER
2,SEKER
3,SEKER
4,SEKER
...,...
13606,DERMASON
13607,DERMASON
13608,DERMASON
13609,DERMASON


In [105]:
# label the features
le = LabelEncoder()
y = le.fit_transform(Y)

#normalize the variation of datas
sc = StandardScaler()
x = sc.fit_transform(X)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [106]:
x[0]

array([-0.84074853, -1.1433189 , -1.30659814, -0.63115304, -1.56505251,
       -2.18572039, -0.84145059, -1.0633406 ,  0.28908744,  0.36761352,
        1.42386706,  1.83911641,  0.68078627,  2.4021726 ,  1.92572347,
        0.83837102])

In [107]:
x

array([[-0.84074853, -1.1433189 , -1.30659814, ...,  2.4021726 ,
         1.92572347,  0.83837102],
       [-0.82918764, -1.01392388, -1.39591111, ...,  3.10089364,
         2.68970162,  0.77113831],
       [-0.80715717, -1.07882906, -1.25235661, ...,  2.23509111,
         1.84135576,  0.91675506],
       ...,
       [-0.37203825, -0.44783294, -0.45047814, ...,  0.28920501,
         0.33632829,  0.39025106],
       [-0.37176543, -0.42702856, -0.42897404, ...,  0.22837456,
         0.2489734 ,  0.03644007],
       [-0.37135619, -0.38755718, -0.2917356 , ..., -0.12777538,
        -0.27648141,  0.71371941]])

In [108]:
y

array([5, 5, 5, ..., 3, 3, 3])

In [109]:
import torch
from torch.utils.data import Dataset, DataLoader

# Example dataset class
class CustomDataset(Dataset):
    def __init__(self, features, targets):
        super(CustomDataset, self).__init__()
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
      #convert feature in tensor
        sample = (torch.tensor(self.features[idx]), torch.tensor(self.targets[idx]))
        return sample

# Assuming you have X_train and y_train as your training data
# Create an instance of CustomDataset
dataset = CustomDataset(x, y)

In [110]:
#split the dataset into 80% train_set,10% validation_set,10% test_set
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [0.8, 0.1, 0.1])

In [111]:
#load train ,validate and test data
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=True)

In [112]:
# autoencoder
class MLP(nn.Module):
    def __init__(self,input_dim, output_features):
        super(MLP, self).__init__()

        # Layers
        self.layers = nn.Sequential(
            nn.Linear(input_dim, int(input_dim*0.5)),
            nn.ReLU(),
            nn.Linear(int(input_dim*0.5), output_features),
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [113]:
#define parameter
batch_size = 32
epochs = 20
learning_rate = 1e-4

In [114]:

#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = MLP(input_dim=16, output_features=7).to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# cross entropy loss
criterion = nn.CrossEntropyLoss()

In [115]:
# training step
for epoch in range(epochs):
    loss = 0
    for features, labels in train_loader :
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        # batch_features = batch_features.view(-1, 784).to(device)
        features = features.to(device).to(torch.float32)
        labels = labels.to(device)
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()

        # compute reconstructions
        outputs = model(features)

        # compute training reconstruction loss
        train_loss = criterion(outputs, labels)

        # compute accumulated gradients
        train_loss.backward()

        # perform parameter update based on current gradients
        optimizer.step()

        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()

    # compute the epoch training loss
    loss = loss / len(train_loader)

    # display the epoch training loss
    print("epoch : {}/{}, recon loss = {:.8f}".format(epoch + 1, epochs, loss))


epoch : 1/20, recon loss = 1.89519440
epoch : 2/20, recon loss = 1.73611337
epoch : 3/20, recon loss = 1.57117347
epoch : 4/20, recon loss = 1.42521659
epoch : 5/20, recon loss = 1.30735809
epoch : 6/20, recon loss = 1.20806134
epoch : 7/20, recon loss = 1.12234705
epoch : 8/20, recon loss = 1.04089010
epoch : 9/20, recon loss = 0.96580271
epoch : 10/20, recon loss = 0.89774615
epoch : 11/20, recon loss = 0.83651445
epoch : 12/20, recon loss = 0.78129257
epoch : 13/20, recon loss = 0.73203578
epoch : 14/20, recon loss = 0.68780351
epoch : 15/20, recon loss = 0.64822901
epoch : 16/20, recon loss = 0.61213531
epoch : 17/20, recon loss = 0.58027674
epoch : 18/20, recon loss = 0.55081843
epoch : 19/20, recon loss = 0.52539382
epoch : 20/20, recon loss = 0.50354476


In [116]:
model.eval()
#model is ready for evaluation

# stop  grad computation
with torch.no_grad():
    num_correct = 0.0
    num_samples = 0.0

    for x, y in test_loader:
        x = x.to(device).to(torch.float32)
        y = y.to(device)

        scores = model(x)

        _, predictions = scores.max(1)

        #prediction where it is equal to actual classification is correct prediction
        num_correct += (predictions == y).sum().item()
        num_samples += y.size(0)

accuracy = float(num_correct / num_samples)
print("Accuracy =", accuracy)


Accuracy = 0.8750918442321822
